In [1]:
import pandas as pd
import numpy as np

# Preprocessing clinical data

We do this prior to upload because of patient merging issues.

In [2]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/raw/ColonGR-PDO_DATA_2024-03-29_1424.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/single_files'
conversion_table_filepath = 'conversion_table/redcap_CRC_conversion_table.csv'
disease_type = 'CRC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = True)

There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
There are no PDO cell lines for record_id 53.
Index(['record_id', 'cell_line_code', 'date_cell_line',
       'redcap_repeat_instrument', 'line_number', 'line_indication', 'CIAH',
       'associated_chip', 'chemotherapy_type', 'targeted_therapy_type',
       'immunotherapy_type', 'first_dose_date', 'last_dose_date',
       'stop_chemotherapy_cause', 'neurotoxicity', 'progression_date',
       'date_RECIST_evaluation', 'progression_type',
       'chemotherapy_RECIST_response', 'PFS', 'sister_cell_line_codes',
       'date_death'],
      dtype='object')
['2022-05-18' '2020-07-03' '2022-11-21' '2020-10-23' '2020-09-01'
 '2023-12-07' '2023-08-26' '2023-04-04' '2023-11-08' '2023-11-27'
 '2023-10-26' '2023-10-25' '2023-12-11' '2023-09-14' '2021-10-09'
 '2022-07-11' '2021-05-15' '2021-12-26' '2021-04-05' '2021-04-24'
 '2022-02-23' '2023-10-05' '2021-06-28' '

In [2]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/raw/PANCREASGR-PDO_DATA_2024-03-29_1423.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/single_files'
conversion_table_filepath = 'conversion_table/redcap_PGR_conversion_table.csv'
disease_type = 'PDAC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = True)

There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 1174.
There are no PDO cell lines for record_id 1174.
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 1174.
There are no PDO cell lines for record_id 1174.
Index(['record_id', 'cell_line_code', 'date_cell_line', 'line_number', 'PFS',
       'chemotherapy_type', 'first_dose_date', 'last_dose_date',
       'stop_chemotherapy_cause', 'progression_date', 'progression_type',
       'chemotherapy_RECIST_res